In [3]:
import torch
from torch.nn import functional as F

In [4]:
x = torch.rand(3)
x

tensor([0.0650, 0.8054, 0.8382])

In [8]:
F.pad(x, (0, 2), value=1)

tensor([0.0650, 0.8054, 0.8382, 1.0000, 1.0000])